In [9]:
import preprocessing as pp
import load_files as lf
import nlp_algorithms as nlp

import pandas as pd
import numpy as np

from sentence_transformers import SentenceTransformer

In [10]:
pd.set_option('display.max_colwidth', -1)
## dfSourceV2 = pd.read_csv('../Dados/v2_semeval-2022_task8_train-data_batch.csv')
train_v1 = pd.read_csv('dados/train v0.1.csv')

train_v1_enen = train_v1[(train_v1['url1_lang'] == 'en') & (train_v1['url2_lang'] == 'en')]

train_v1_enen.head()

C:\Users\lucas\AppData\Local\Temp/ipykernel_8908/2099918564.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


,url1_lang,url2_lang,pair_id,link1,link2,ia_link1,ia_link2,Geography,Entities,Time,Narrative,Overall,Style,Tone
0,en,en,1484084337_1484110209,https://www.washingtonpost.com/local/virginia-man-arrested-in-fatal-dui-crash-in-west-virginia/2020/01/01/740fbc7a-2cbe-11ea-bffe-020c88b3f120_story.html,https://www.washingtonpost.com/world/the_americas/haitis-leader-marks-independence-day-amid-security-concerns/2020/01/01/dc4033a4-2cc5-11ea-bffe-020c88b3f120_story.html,https://web.archive.org/web/www.washingtonpost.com/local/virginia-man-arrested-in-fatal-dui-crash-in-west-virginia/2020/01/01/740fbc7a-2cbe-11ea-bffe-020c88b3f120_story.html,https://web.archive.org/web/www.washingtonpost.com/world/the_americas/haitis-leader-marks-independence-day-amid-security-concerns/2020/01/01/dc4033a4-2cc5-11ea-bffe-020c88b3f120_story.html,4.0,4.000000,1.000000,4.000000,4.000000,1.666667,2.000000
1,en,en,1484396422_1483924666,https://www.stlucianewsonline.com/guyana-three-injured-after-car-crashes-into-utility-pole/,https://www.thestar.com/news/world/europe/2020/01/01/fire-kills-animals-at-zoo-in-western-germany.html,https://web.archive.org/web/www.stlucianewsonline.com/guyana-three-injured-after-car-crashes-into-utility-pole/,https://web.archive.org/web/www.thestar.com/news/world/europe/2020/01/01/fire-kills-animals-at-zoo-in-western-germany.html,4.0,4.000000,1.000000,4.000000,3.666667,1.666667,1.333333
2,en,en,1484698254_1483758694,https://www.teaparty.org/trump-brings-in-2020-at-mar-a-lago-were-going-to-have-a-great-year-423052/,https://www.timesofisrael.com/trump-says-he-does-not-expect-war-with-iran-likes-peace/,https://web.archive.org/web/www.teaparty.org/trump-brings-in-2020-at-mar-a-lago-were-going-to-have-a-great-year-423052/,https://web.archive.org/web/www.timesofisrael.com/trump-says-he-does-not-expect-war-with-iran-likes-peace/,1.0,2.000000,1.000000,2.333333,2.333333,1.000000,1.333333
3,en,en,1576314516_1576455088,https://gadgets.ndtv.com/apps/news/zomato-uber-eats-business-acquisition-india-all-stock-deal-2167155,https://gadgets.ndtv.com/internet/news/indian-online-food-delivery-market-to-hit-usd-8-billion-by-2020-google-bcg-report-2171043,https://web.archive.org/web/gadgets.ndtv.com/apps/news/zomato-uber-eats-business-acquisition-india-all-stock-deal-2167155,https://web.archive.org/web/gadgets.ndtv.com/internet/news/indian-online-food-delivery-market-to-hit-usd-8-billion-by-2020-google-bcg-report-2171043,1.0,2.333333,2.666667,1.666667,2.000000,1.666667,1.666667
4,en,en,1484036253_1483894099,https://news.yahoo.com/india-approves-third-moon-mission-085759387.html,https://www.channelnewsasia.com/news/asia/india-targets-new-moon-mission-in-2020-12225344,https://web.archive.org/web/news.yahoo.com/india-approves-third-moon-mission-085759387.html,https://web.archive.org/web/www.channelnewsasia.com/news/asia/india-targets-new-moon-mission-in-2020-12225344,1.0,1.250000,1.000000,1.250000,1.250000,1.000000,1.000000


In [11]:
data_path = 'dados/train v0.1/'

lista_docs = []
lista_error = []
lista_vazio = []
values = train_v1_enen[['pair_id', 'Overall']]

for index, values in values.iterrows():
    
    try:
        
        json_pair = lf.get_json_document_pair(data_path, values['pair_id'])
        text_doc1 = json_pair[0]['text']
        text_doc2 = json_pair[1]['text']
        
        if ( len(text_doc1) > 0 ) and ( len(text_doc2) > 0 ):
            lista_docs.append((values['pair_id'], text_doc1, text_doc2, values['Overall']))
        else:
            lista_vazio.append(values['pair_id'])
    
    except:
        lista_error.append(values['pair_id'])

In [12]:
df_text = pd.DataFrame(lista_docs,  columns=['pair_id', 'doc1', 'doc2', 'Overall'])
#df_text.head(1)

In [13]:
df_text.shape

(1357, 4)

## Pre-processamento

In [14]:
pre_processing_list = [
    {},
    {"basic_processing": True},
    {"basic_processing": True, "no_stopwords": True}
    ]

'''pre_processing_list = [
    {},
    {"basic_processing": True},
    {"basic_processing": True, "no_stopwords": True},
    {"basic_processing": True, "stemming": True},
    {"basic_processing": True, "no_stopwords": True, "stemming": True},
    {"basic_processing": True, "lema":  True},
    {"basic_processing": True, "no_stopwords": True, "lema": True}]'''

df_pp = pp.pre_process_all(df_text, pre_processing_list)
#df_pp = pp.pre_process_all(df_text.iloc[:3], pre_processing_list)

# Algoritmos

In [7]:
df_results = df_pp[["pair_id", "Overall"]]
df_results.head(1)

,pair_id,Overall
0,1484084337_1484110209,4.0


## BERT

In [8]:
pre_processing_list

[{},
 {'basic_processing': True},
 {'basic_processing': True, 'no_stopwords': True}]

In [9]:
#df_pp.columns

#df_pp[['doc2_pipeline0', 'Overall']]

### bert-base-nli-mean-tokens

In [10]:
'''## Funcoes do encode pro fine tune
import tensorflow as tf
import tensorflow_addons as tfa

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy

from transformers import TFBertForSequenceClassification
from transformers import BertTokenizer

# can be up to 512 for BERT
MAX_LENGTH = 256
BATCH_SIZE = 64
SEED = 42
from transformers import BertTokenizer

tokenizer = SentenceTransformer('sentence-transformers/bert-base-nli-mean-tokens')
#BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased', do_lower_case = False)

def map_example_to_dict(input_ids, attention_masks, token_type_ids, label):
  return {
      "input_ids": input_ids,
      "token_type_ids": token_type_ids,
      "attention_mask": attention_masks,
  }, label

def convert_example_to_feature(titulo_1, titulo_2):
    return tokenizer.encode_plus(titulo_1, titulo_2,
                                 add_special_tokens = True, # adiciona [CLS], [SEP]
                                 max_length = MAX_LENGTH, # comprimento máximo do texto de entrada
                                 padding = 'max_length', # adiciona [PAD] até o tam_max (MAX_LENGTH)
                                 truncation = True, # padrão = 'longest_first'
                                 return_attention_mask = True, # adiciona máscara de atenção para não focar nos tokens do pad
                                )
                        
def encode_examples(df_titulos, labels, limit = -1):
    
    # prepare list, so that we can build up final TensorFlow dataset from slices.
    input_ids_list = []
    token_type_ids_list = []
    attention_mask_list = []
    label_list = []
    
    if (limit > 0):
        ds = ds.take(limit)
    
    # for review, label in tfds.as_numpy(ds):
    for titulo_1, titulo_2, label in zip(df_titulos["titulo_1"], df_titulos["titulo_2"], labels):
        
        bert_input = convert_example_to_feature(titulo_1, titulo_2)
        input_ids_list.append(bert_input['input_ids'])
        token_type_ids_list.append(bert_input['token_type_ids'])
        attention_mask_list.append(bert_input['attention_mask'])
        label_list.append([label])
        
    return tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list, token_type_ids_list, label_list)).map(map_example_to_dict)

def get_bert_data(X_train, y_train, X_valid, y_valid):
    
    # train dataset
    ds_train = encode_examples(X_train, y_train).batch(BATCH_SIZE)

    #validation dataset
    ds_valid = encode_examples(X_valid, y_valid).batch(BATCH_SIZE)

    return ds_train, ds_valid

import pandas as pd
import numpy as np
import time

from sklearn.metrics.pairwise import cosine_similarity

from scipy import spatial
from sklearn.model_selection import train_test_split

###Bert###
def get_bert(model, doc1, doc2):
    
    data = [doc1, doc2]
    sentence_embeddings = model.encode(data)

    infer1 = sentence_embeddings[0]
    infer2 = sentence_embeddings[1]
    
    cos_similarity = 1 - spatial.distance.cosine(infer1, infer2) #de 0 a 1
    
    return cos_similarity



def apply_bert(df, len_pipeline, model, model_name, fine_tune = False):

    time_list = []

    df_bert = pd.DataFrame()
    for index in range(len_pipeline):
        if fine_tune == True:
            X = df[f'doc1_pipeline{index}']
            y = df['Overall']
            X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2, random_state = SEED, stratify = y)

            learning_rate = 2e-5
            number_of_epochs = 3
            ds_train, ds_valid, ds_test = get_bert_data(X_train, y_train, X_valid, y_valid)
            
            # model initialization
            #model = TFBertForSequenceClassification.from_pretrained('neuralmind/bert-base-portuguese-cased', from_pt = True)

            # choosing Adam optimizer
            optimizer = Adam(learning_rate=learning_rate, epsilon = 1e-08)
            loss = SparseCategoricalCrossentropy(from_logits = True)
            metric_acc = SparseCategoricalAccuracy('accuracy')
            model.compile(optimizer = optimizer, loss = loss, metrics = [metric_acc])

            #Training model
            bert_history = model.fit(ds_train, epochs = number_of_epochs, validation_data = ds_valid)
        start_time = time.time()
        df_bert[f'bert_{model_name}{index}'] = df.apply(lambda row: get_bert(model, " ".join(row[f'doc1_pipeline{index}']), " ".join(row[f'doc2_pipeline{index}'])), axis=1)
        time_list.append((f'bert_{model_name}{index}', time.time()-start_time))

    return (df_bert, pd.DataFrame(time_list))'''

'## Funcoes do encode pro fine tune\nimport tensorflow as tf\nimport tensorflow_addons as tfa\n\nfrom tensorflow.keras.optimizers import Adam\nfrom tensorflow.keras.losses import SparseCategoricalCrossentropy\nfrom tensorflow.keras.metrics import SparseCategoricalAccuracy\n\nfrom transformers import TFBertForSequenceClassification\nfrom transformers import BertTokenizer\n\n# can be up to 512 for BERT\nMAX_LENGTH = 256\nBATCH_SIZE = 64\nSEED = 42\nfrom transformers import BertTokenizer\n\ntokenizer = SentenceTransformer(\'sentence-transformers/bert-base-nli-mean-tokens\')\n#BertTokenizer.from_pretrained(\'neuralmind/bert-base-portuguese-cased\', do_lower_case = False)\n\ndef map_example_to_dict(input_ids, attention_masks, token_type_ids, label):\n  return {\n      "input_ids": input_ids,\n      "token_type_ids": token_type_ids,\n      "attention_mask": attention_masks,\n  }, label\n\ndef convert_example_to_feature(titulo_1, titulo_2):\n    return tokenizer.encode_plus(titulo_1, titulo_2

In [11]:
id = df_pp['pair_id']
overall = df_pp['Overall']

pd.concat([id, overall], axis=1)

,pair_id,Overall
0,1484084337_1484110209,4.000000
1,1484396422_1483924666,3.666667
2,1484698254_1483758694,2.333333
3,1576314516_1576455088,2.000000
4,1484036253_1483894099,1.250000
...,...,...
1352,1484430020_1483808942,1.000000
1353,1562189733_1562214022,1.000000
1354,1490711494_1500549122,4.000000
1355,1559851360_1553590552,1.000000


In [12]:
'''from sklearn.model_selection import train_test_split

train_samples = []
valid_samples = []

index=0

X = df_pp[[f'doc1_pipeline{index}', f'doc2_pipeline{index}']]
y = df_pp['Overall']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2, random_state = SEED)

train_df = pd.concat([X_train, y_train], axis=1)
train_df = train_df.reset_index()
valid_df = pd.concat([X_valid, y_valid], axis=1)
valid_df = valid_df.reset_index()


for i in range(1, len(train_df)):
    inp_example = InputExample(texts=[train_df.iloc[i][f'doc1_pipeline{index}'], train_df.iloc[i][f'doc2_pipeline{index}']], label=train_df.iloc[i]['Overall'])
    train_samples.append(inp_example)

for j in range(1, len(valid_df)):
    inp_example = InputExample(texts=[valid_df.iloc[j][f'doc1_pipeline{index}'], valid_df.iloc[j][f'doc2_pipeline{index}']], label=valid_df.iloc[j]['Overall'])
    valid_samples.append(inp_example)'''

"from sklearn.model_selection import train_test_split\n\ntrain_samples = []\nvalid_samples = []\n\nindex=0\n\nX = df_pp[[f'doc1_pipeline{index}', f'doc2_pipeline{index}']]\ny = df_pp['Overall']\n\nX_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2, random_state = SEED)\n\ntrain_df = pd.concat([X_train, y_train], axis=1)\ntrain_df = train_df.reset_index()\nvalid_df = pd.concat([X_valid, y_valid], axis=1)\nvalid_df = valid_df.reset_index()\n\n\nfor i in range(1, len(train_df)):\n    inp_example = InputExample(texts=[train_df.iloc[i][f'doc1_pipeline{index}'], train_df.iloc[i][f'doc2_pipeline{index}']], label=train_df.iloc[i]['Overall'])\n    train_samples.append(inp_example)\n\nfor j in range(1, len(valid_df)):\n    inp_example = InputExample(texts=[valid_df.iloc[j][f'doc1_pipeline{index}'], valid_df.iloc[j][f'doc2_pipeline{index}']], label=valid_df.iloc[j]['Overall'])\n    valid_samples.append(inp_example)"

In [13]:
import pandas as pd
import numpy as np
import time

from sklearn.metrics.pairwise import cosine_similarity

from scipy import spatial

from sentence_transformers import evaluation
from sentence_transformers import SentenceTransformer, SentencesDataset, InputExample, losses
from torch.utils.data import DataLoader

from sklearn.model_selection import train_test_split

SEED = 42


###Bert###
def get_bert(model, doc1, doc2):
    
    data = [doc1, doc2]
    sentence_embeddings = model.encode(data)

    infer1 = sentence_embeddings[0]
    infer2 = sentence_embeddings[1]
    
    cos_similarity = 1 - spatial.distance.cosine(infer1, infer2) #de 0 a 1
    
    return cos_similarity

def apply_bert(df, len_pipeline, model, model_name, fine_tune = False):

    train_batch_size = 16
    num_epochs = 4
    model_save_path = 'output/training_stsbenchmark_continue_training-'+model_name

    time_list = []

    df_bert = pd.DataFrame()
    for index in range(len_pipeline):
        if fine_tune==True:

            train_samples = []
            valid_samples = []

            X = df[[f'doc1_pipeline{index}', f'doc2_pipeline{index}']]
            y = df['Overall']

            X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2, random_state = SEED)

            train_df = pd.concat([X_train, y_train], axis=1)
            train_df = train_df.reset_index()
            valid_df = pd.concat([X_valid, y_valid], axis=1)
            valid_df = valid_df.reset_index()


            ### TALVEZ: Na hora de montar isso aqui n possa passarr tokenizado? 
            ### Tenho q testar sem tokenizar
            for i in range(1, len(train_df)):
                inp_example = InputExample(texts=[train_df.iloc[i][f'doc1_pipeline{index}'], train_df.iloc[i][f'doc2_pipeline{index}']], label=float(train_df.iloc[i]['Overall']))
                train_samples.append(inp_example)

            for j in range(1, len(valid_df)):
                inp_example = InputExample(texts=[valid_df.iloc[j][f'doc1_pipeline{index}'], valid_df.iloc[j][f'doc2_pipeline{index}']], label=float(valid_df.iloc[j]['Overall']))
                valid_samples.append(inp_example)

            train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)
            train_loss = losses.CosineSimilarityLoss(model=model)
            evaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(valid_samples)#(sts_reader.get_examples('sts-dev.csv'))

            model.fit(train_objectives=[(train_dataloader, train_loss)],
                    evaluator=evaluator,
                    epochs=num_epochs,
                    evaluation_steps=1000,
                    #warmup_steps=warmup_steps, 100?
                    output_path=model_save_path
                    )

        start_time = time.time()
        df_bert[f'bert_{model_name}{index}'] = df.apply(lambda row: get_bert(model, " ".join(row[f'doc1_pipeline{index}']), " ".join(row[f'doc2_pipeline{index}'])), axis=1)
        time_list.append((f'bert_{model_name}{index}', time.time()-start_time))

    return (df_bert, pd.DataFrame(time_list))

In [14]:
df_pp[:50].shape

(50, 10)

In [15]:
#model = SentenceTransformer('sentence-transformers/bert-base-nli-mean-tokens')

#model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

model_name = 'bert_miniLM'


(df_bert, time_bert) = apply_bert(df_pp, len(pre_processing_list), model, model_name, fine_tune=True)

df_results = df_results.join(df_bert)

Downloading: 100%|██████████| 1.18k/1.18k [00:00<00:00, 582kB/s]
Downloading: 100%|██████████| 190/190 [00:00<00:00, 93.7kB/s]
Downloading: 100%|██████████| 10.6k/10.6k [00:00<00:00, 2.12MB/s]
Downloading: 100%|██████████| 116/116 [00:00<00:00, 28.8kB/s]
Downloading: 100%|██████████| 39.3k/39.3k [00:00<00:00, 295kB/s] 
Downloading: 100%|██████████| 349/349 [00:00<00:00, 116kB/s]
Downloading: 100%|██████████| 438M/438M [00:13<00:00, 32.2MB/s] 
Downloading: 100%|██████████| 53.0/53.0 [00:00<00:00, 13.3kB/s]
Downloading: 100%|██████████| 239/239 [00:00<00:00, 240kB/s]
Downloading: 100%|██████████| 466k/466k [00:00<00:00, 751kB/s] 
Downloading: 100%|██████████| 363/363 [00:00<00:00, 73.4kB/s]
Downloading: 100%|██████████| 13.1k/13.1k [00:00<00:00, 6.69MB/s]
Downloading: 100%|██████████| 232k/232k [00:00<00:00, 431kB/s] 
Epoch: 100%|██████████| 4/4 [08:54<00:00, 133.65s/it]


In [16]:
df_bert.to_csv('./resultados/bert_minilm_ft.csv', index = False)
time_bert.to_csv('./resultados/bert_minilm_ft_time.csv', index = False)

In [19]:
df_results.corr()

,Overall,bert_bert_miniLM0,bert_bert_miniLM1,bert_bert_miniLM2
Overall,1.000000,-0.809337,-0.794582,-0.631725
bert_bert_miniLM0,-0.809337,1.000000,0.982540,0.811701
bert_bert_miniLM1,-0.794582,0.982540,1.000000,0.835261
bert_bert_miniLM2,-0.631725,0.811701,0.835261,1.000000


In [18]:
a

NameError: name 'a' is not defined

In [ ]:
df_results.corr() RESULTADO SEM FINE TUNE DO MINILM

,Overall,bert_base_mean0,bert_base_mean1,bert_base_mean2
Overall,1.000000,-0.813552,-0.811477,-0.774372
bert_base_mean0,-0.813552,1.000000,0.985662,0.941072
bert_base_mean1,-0.811477,0.985662,1.000000,0.956074
bert_base_mean2,-0.774372,0.941072,0.956074,1.000000


### bert-base-nli-max-tokens

### all-MiniLM-L6-v2

## USE (Universal Sentence Encoder)

In [2]:
from absl import logging

import tensorflow as tf

import tensorflow_hub as hub
import numpy as np
import os
import pandas as pd
import re

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


#### sts

In [40]:
sts_data = df_pp[['Overall', 'doc1', 'doc2']]#sts_dev

def run_sts_benchmark(batch):
  sts_encode1 = tf.nn.l2_normalize(embed(tf.constant(batch['doc1'].tolist())), axis=1)
  sts_encode2 = tf.nn.l2_normalize(embed(tf.constant(batch['doc2'].tolist())), axis=1)
  cosine_similarities = tf.reduce_sum(tf.multiply(sts_encode1, sts_encode2), axis=1)
  clip_cosine_similarities = tf.clip_by_value(cosine_similarities, -1.0, 1.0)
  scores = 1.0 - tf.acos(clip_cosine_similarities) / math.pi
  """Returns the similarity scores"""
  return scores

dev_scores = sts_data['Overall'].tolist()
scores = []
for batch in np.array_split(sts_data, 10):
  scores.extend(run_sts_benchmark(batch))

pearson_correlation = scipy.stats.pearsonr(scores, dev_scores)
print('Pearson correlation coefficient = {0}\np-value = {1}'.format(
    pearson_correlation[0], pearson_correlation[1]))

Pearson correlation coefficient = -0.7655629969170454
p-value = 8.087010431362859e-262
